# Álgebra Linear Computacional - CKP8122 - MDCC - UFC
### Francisco Mateus dos Anjos Silva
# Decomposição SVD

Em álgebra linear, a **decomposição em valores singulares** ou singular value decomposition (SVD) é a fatoração de uma matriz real ou complexa, com diversas aplicações importantes em processamento de sinais e estatística.

Formalmente, a decomposição em valores singulares de uma matriz m×n real ou complexa $M$ é uma fatoração ou fatorização na forma:

${\displaystyle M=U\Sigma V^{*}}$,

onde U é uma matriz unitária m×m real ou complexa, Σ é uma matriz retangular diagonal m×n com números reais não-negativos na diagonal, e V* (a conjugada transposta de V) é uma matriz unitária n×n real ou complexa. 
As entradas diagonais Σi,i de Σ são os chamados valores singulares de M. As m colunas de U e as n colunas de V são os chamados vetores singulares à esquerda e vetores singulares à direita de M, respetivamente.

A decomposição em valores singulares e a decomposição em autovalores são intimamente relacionadas. Mais especificamente:

- Os vetores singulares à esquerda de M são autovetores de {\displaystyle MM^{*}.}MM^{*}.
- Os vetores singulares à direita de M são autovetores de {\displaystyle M^{*}M.}M^{*}M.
- Os valores singulares não-nulos de M (ao longo da diagonal de Σ) são as raízes quadradas dos autovalores não-nulos de ${\displaystyle M^{*}M}$ ou ${\displaystyle MM^{*}}$.

Dentre as aplicações que envolvem a SVD estão o cálculo da pseudoinversa, o ajuste (*fitting*) de dados por mínimos quadrados, aproximação de matrizes, e a determinação do posto, imagem e núcleo de uma matriz.

**Referências:**

- https://pt.wikipedia.org/wiki/Decomposi%C3%A7%C3%A3o_em_valores_singulares

In [1]:
# Implementar o a decomposição SVD de uma matriz qualquer A mxn

# 1) Entrar com a matriz
# 2) Encontrar e imprimir as matrizes U, Sigma e V
# 3) Mostrar que o produto U . Sigma . V^T  = A

# Obs.: Quando necessário, use extensão de base e ortogonalização de Gram-Schmidt para encontrar U ou V completas.

In [2]:
import numpy as np
import math
from functions_for_svd import nullspace, gram_schmidt, normalize_basis, qr_method_eigvalues_eigvectors

In [3]:
# Estender a base
def extended_basis(A):
    n_A = len(A)
    m_A = len(A[0])
    
    basis = np.array([A[0]])
    for i in range(1, n_A):
        basis = np.append(arr=basis, values=[A[i]], axis=0)

    nullspace_a = nullspace(A)
    nullspace_a = normalize_basis(nullspace_a)

    if not nullspace_a is None:
        basis = np.append(arr=basis, values=nullspace_a, axis=0)
    return basis

In [4]:
def svd_decomposition(A):
    n_A = len(A)
    m_A = len(A[0])
    AT_A = A.T @ A
    D, P = qr_method_eigvalues_eigvectors(AT_A)
    
    V = P
    U = None 
    sigma = np.zeros((n_A, m_A))
    
    for i in range(0, len(sigma)):
        if i < len(D) and D[i,i] != 0:
            sigma[i,i] = np.sqrt(D[i,i])
    
    if n_A < m_A:
        sigma_inverse = sigma.T.copy()
        
        for j in range(len(sigma)):
            if j < len(sigma[0]):
                sigma_inverse[j,j] = 1 / sigma_inverse[j,j]
                
        U = A @ V @ sigma_inverse
           
    if n_A >= m_A:
        U = A @ V
        
        for j in range(0,len(U[0])):
            if j < len(sigma) and sigma[j,j] != 0:
                U[:,j] = U[:,j] * (1/sigma[j,j])
                
        if n_A > m_A:
            U = extended_basis(U.T).T
            
    return U, sigma, V.T

### Teste com matriz n < m

In [8]:
np.set_printoptions(precision=4)

# Matriz 4x5
A = np.array([[3,-2, 1,-1, 2],
              [1, 3, 0, 2, 1],
              [5, 0, 2, 0, 2],
              [1, 5, 0, 3, 0]])

print('\nMatriz A:\n', A)

U, sigma, V_T = svd_decomposition(A)
U_numpy, sigma_numpy_v, V_T_numpy = np.linalg.svd(A)

sigma_numpy = np.zeros((len(A), len(A[0])), dtype=np.float32)
for i in range(0, len(sigma_numpy_v)):
    sigma_numpy[i,i] = sigma_numpy_v[i]

print('\nV.T:\n', V_T)
print('\nV.T (numpy):\n', V_T_numpy)

print('\nSigma:\n', sigma)
print('\nSigma (numpy):\n', sigma_numpy)

print('\nU:\n', U)
print('\nU (numpy):\n', U_numpy)

print('\nMatriz A:\n', A)
print('\nVerificação A = U * Sigma * V.T:\n', (U @ sigma @ V_T))
print('\nVerificação A = U * Sigma * V.T (numpy):\n', (U_numpy @ sigma_numpy @ V_T_numpy))


Matriz A:
 [[ 3 -2  1 -1  2]
 [ 1  3  0  2  1]
 [ 5  0  2  0  2]
 [ 1  5  0  3  0]]

V.T:
 [[ 7.7439e-01  3.8452e-01  2.5332e-01  2.5086e-01  3.5404e-01]
 [-3.6566e-01  7.6210e-01 -1.8810e-01  4.5216e-01 -2.1367e-01]
 [ 2.6422e-01  1.0248e-01  4.0804e-01 -1.8952e-01 -8.4692e-01]
 [-4.4360e-01  1.0409e-01  8.4033e-01 -4.7661e-02  2.8975e-01]
 [-3.9433e-06  5.0000e-01 -1.6667e-01 -8.3333e-01  1.6667e-01]]

V.T (numpy):
 [[ 3.1265e-01  8.0257e-01  5.5288e-02  4.9269e-01  1.1105e-01]
 [ 7.9728e-01 -2.9077e-01  3.1065e-01 -1.5693e-01  3.9833e-01]
 [ 2.6421e-01  1.0248e-01  4.0806e-01 -1.8950e-01 -8.4691e-01]
 [-4.4360e-01  1.0409e-01  8.4033e-01 -4.7661e-02  2.8975e-01]
 [ 1.9329e-15  5.0000e-01 -1.6667e-01 -8.3333e-01  1.6667e-01]]

Sigma:
 [[7.117  0.     0.     0.     0.    ]
 [0.     7.0997 0.     0.     0.    ]
 [0.     0.     0.9626 0.     0.    ]
 [0.     0.     0.     0.1235 0.    ]]

Sigma (numpy):
 [[7.2541 0.     0.     0.     0.    ]
 [0.     6.9596 0.     0.     0.    ]
 [0.  

### Teste com matriz n = m

In [6]:
np.set_printoptions(precision=4)

# Matriz 4x4
A = np.array([[3,-2, 1,-1],
              [1, 3, 0, 2],
              [5, 0, 2, 0],
              [1, 5, 0, 3]])

print('\nMatriz A:\n', A)

U, sigma, V_T = svd_decomposition(A)
U_numpy, sigma_numpy_v, V_T_numpy = np.linalg.svd(A)

sigma_numpy = np.zeros((len(A), len(A[0])), dtype=np.float32)
for i in range(0, len(sigma_numpy_v)):
    sigma_numpy[i,i] = sigma_numpy_v[i]

print('\nV.T:\n', V_T)
print('\nV.T (numpy):\n', V_T_numpy)

print('\nSigma:\n', sigma)
print('\nSigma (numpy):\n', sigma_numpy)

print('\nU:\n', U)
print('\nU (numpy):\n', U_numpy)

print('\nMatriz A:\n', A)
print('\nVerificação A = U * Sigma * V.T:\n', (U @ sigma @ V_T))
print('\nVerificação A = U * Sigma * V.T (numpy):\n', (U_numpy @ sigma_numpy @ V_T_numpy))


Matriz A:
 [[ 3 -2  1 -1]
 [ 1  3  0  2]
 [ 5  0  2  0]
 [ 1  5  0  3]]

V.T:
 [[ 0.1818  0.8396  0.0053  0.5118]
 [-0.9212  0.156  -0.3486  0.0748]
 [-0.2737  0.3355  0.7745 -0.4612]
 [-0.2086 -0.3977  0.5278  0.721 ]]

V.T (numpy):
 [[-0.1642 -0.8424  0.0013 -0.5131]
 [ 0.9244 -0.1401  0.3487 -0.065 ]
 [ 0.2737 -0.3355 -0.7745  0.4612]
 [ 0.2086  0.3977 -0.5278 -0.721 ]]

Sigma:
 [[7.2387 0.     0.     0.    ]
 [0.     6.3611 0.     0.    ]
 [0.     0.     0.367  0.    ]
 [0.     0.     0.     0.0592]]

Sigma (numpy):
 [[7.239  0.     0.     0.    ]
 [0.     6.3608 0.     0.    ]
 [0.     0.     0.367  0.    ]
 [0.     0.     0.     0.0592]]

U:
 [[-0.2266 -0.5501 -0.6989 -0.3988]
 [ 0.5145 -0.0477 -0.5166  0.6826]
 [ 0.127  -0.8337  0.4915  0.2146]
 [ 0.8172  0.0131  0.0551 -0.5736]]

U (numpy):
 [[ 0.2358  0.5451  0.6989  0.3986]
 [-0.5136  0.0588  0.5166 -0.6825]
 [-0.1131  0.8363 -0.4916 -0.2149]
 [-0.8172  0.0046 -0.0551  0.5737]]

Matriz A:
 [[ 3 -2  1 -1]
 [ 1  3  0  2]
 [ 5 

### Teste com matriz n > m

In [7]:
np.set_printoptions(precision=4)

# Matriz 5x4
A = np.array([[3,-2, 1,-1],
              [1, 3, 0, 2],
              [5, 0, 2, 0],
              [1, 5, 0, 3],
              [3, 7, 1, 2]])

print('\nMatriz A:\n', A)

U, sigma, V_T = svd_decomposition(A)
U_numpy, sigma_numpy_v, V_T_numpy = np.linalg.svd(A)

sigma_numpy = np.zeros((len(A), len(A[0])), dtype=np.float32)
for i in range(0, len(sigma_numpy_v)):
    sigma_numpy[i,i] = sigma_numpy_v[i]

print('\nV.T:\n', V_T)
print('\nV.T (numpy):\n', V_T_numpy)

print('\nSigma:\n', sigma)
print('\nSigma (numpy):\n', sigma_numpy)

print('\nU:\n', U)
print('\nU (numpy):\n', U_numpy)

print('\nMatriz A:\n', A)
print('\nVerificação A = U * Sigma * V.T:\n', (U @ sigma @ V_T))
print('\nVerificação A = U * Sigma * V.T (numpy):\n', (U_numpy @ sigma_numpy @ V_T_numpy))


Matriz A:
 [[ 3 -2  1 -1]
 [ 1  3  0  2]
 [ 5  0  2  0]
 [ 1  5  0  3]
 [ 3  7  1  2]]

V.T:
 [[-0.3557 -0.8552 -0.0967 -0.3644]
 [ 0.8636 -0.3162  0.3426 -0.1918]
 [ 0.1157 -0.4079 -0.1701  0.8895]
 [-0.3381 -0.0476  0.9189  0.1978]]

V.T (numpy):
 [[-0.3555 -0.8553 -0.0966 -0.3645]
 [ 0.8637 -0.316   0.3427 -0.1917]
 [ 0.1157 -0.4079 -0.1701  0.8895]
 [ 0.3381  0.0476 -0.9189 -0.1978]]

Sigma:
 [[10.6249  0.      0.      0.    ]
 [ 0.      6.4154  0.      0.    ]
 [ 0.      0.      1.3716  0.    ]
 [ 0.      0.      0.      0.2637]
 [ 0.      0.      0.      0.    ]]

Sigma (numpy):
 [[10.625   0.      0.      0.    ]
 [ 0.      6.4154  0.      0.    ]
 [ 0.      0.      1.3716  0.    ]
 [ 0.      0.      0.      0.2637]
 [ 0.      0.      0.      0.    ]]

U:
 [[ 0.0857  0.5857  0.0753 -0.7505 -0.2841]
 [-0.3435 -0.0731  0.4892 -0.3232  0.7296]
 [-0.1856  0.7799  0.1737  0.5589  0.1211]
 [-0.5388 -0.2015  0.5429  0.0659 -0.6087]
 [-0.7415  0.0524 -0.6558 -0.1247  0.041 ]]

U (numpy